In [7]:
import argparse
import gym
from gym import wrappers
import os.path as osp
import random
import numpy as np
import itertools
import tensorflow as tf
import tensorflow.contrib.layers as layers

import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
# from utils import plotting
from arm_env_dqn import ArmEnvDQN
from option_class import option

In [8]:
def arm_model(img_in, num_actions, scope, reuse=False):
    # as described in https://storage.googleapis.com/deepmind-data/assets/papers/DeepMindNature14236Paper.pdf
    with tf.variable_scope(scope, reuse=reuse):
        out = img_in
        with tf.variable_scope("convnet"):
            # original architecture
            out = layers.convolution2d(out, num_outputs=32, kernel_size=8, stride=4, activation_fn=tf.nn.relu)
            out = layers.convolution2d(out, num_outputs=64, kernel_size=4, stride=2, activation_fn=tf.nn.relu)
            out = layers.convolution2d(out, num_outputs=64, kernel_size=3, stride=1, activation_fn=tf.nn.relu)
        out = layers.flatten(out)
        with tf.variable_scope("action_value"):
            out = layers.fully_connected(out, num_outputs=256,         activation_fn=tf.nn.relu)
            out = layers.fully_connected(out, num_outputs=num_actions, activation_fn=None)

        return out

In [10]:
def encode_observation(frame):
    img_h, img_w = frame.shape[1], frame.shape[2]
    return frame.transpose(1, 2, 0, 3).reshape(img_h, img_w, -1)

def stop_cond(state):
    """Checks if this state terminates the option"""
    return False

def main():
    env = ArmEnvDQN(episode_max_length=200,
                 size_x=6,
                 size_y=4,
                 cubes_cnt=4,
                 scaling_coeff=3,
                 action_minus_reward=-1,
                 finish_reward=100,
                 tower_target_size=4)
    
    def stop_cond1(env):
        if env._arm_x+1 < env._size_x:
            if env._grid[env._arm_x+1, env._arm_y] == 1 and env._arm_x+2 >= env._size_x:
                return True
            if env._grid[env._arm_x+1, env._arm_y] == 1 and env._arm_x+2 < env._size_x:
                if env._grid[env._arm_x+2, env._arm_y] == 1:
                    return True
        else:
            return True
        return False
    
    def stop_cond2(env):
        if env._arm_x == 0 and env._grid[1, env._arm_y] == 1 and env._grid[2, env._arm_y] == 0:
            return True
        return False
        
    
    option1 = option(env, stop_cond2, path = "option2/dqn_graph.ckpt", import_scope = "option2")
    next_obs, opt_reward, done, opt_length, info = option1.step(env)
    print(opt_reward, done, opt_length, info)
#     env.render()


if __name__ == "__main__":
    main()

INFO:tensorflow:Restoring parameters from option2/dqn_graph.ckpt


[2018-05-14 21:14:00,086] Restoring parameters from option2/dqn_graph.ckpt



[[3 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [1 1 1 1]]

[[2 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [1 1 1 1]]

[[0 0 0 0]
 [2 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [1 1 1 1]]

[[0 0 0 0]
 [0 0 0 0]
 [2 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [1 1 1 1]]

[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [2 0 0 0]
 [0 0 0 0]
 [1 1 1 1]]

[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [2 0 0 0]
 [1 1 1 1]]

[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [2 0 0 0]
 [1 0 0 0]
 [0 1 1 1]]

[[0 0 0 0]
 [0 0 0 0]
 [2 0 0 0]
 [1 0 0 0]
 [0 0 0 0]
 [0 1 1 1]]

[[0 0 0 0]
 [2 0 0 0]
 [1 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 1 1 1]]

[[2 0 0 0]
 [1 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 1 1 1]]
-9 False 9 None


In [ ]:
! tensorboard --logdir=logs_g2

In [ ]:
a = "fdjdjdjjd"
b = "skks"

In [ ]:
a+b